In [1]:
import numpy as np 
import pandas as pd
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score

In [2]:
df=pd.read_csv("../DataSets/Nifty_Stocks.csv")

In [3]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,Category,Daily_Return,Price_Range,Volatility,Cumulative_Return,Average_Price
0,2024-01-01,3790.000000,3832.000000,3773.000000,3811.100098,3757.422119,825907,TCS,IT_industry,3711.100098,59.000000,3732.999714,NaN,5695.550049
1,2024-01-02,3811.100098,3811.100098,3767.250000,3783.199951,3729.915039,1344068,TCS,IT_industry,3683.199951,43.850098,3711.521697,-0.007321,5702.700073
2,2024-01-03,3767.000000,3771.850098,3687.050049,3691.750000,3639.753174,1803075,TCS,IT_industry,3591.750000,84.800049,3671.977407,-0.031493,5612.875000
3,2024-01-04,3701.750000,3719.000000,3651.000000,3666.800049,3615.154541,3598144,TCS,IT_industry,3566.800049,68.000000,3619.430895,-0.038252,5535.150024
4,2024-01-05,3675.000000,3747.750000,3674.850098,3737.899902,3685.252930,1963127,TCS,IT_industry,3637.899902,72.899902,3649.436771,-0.018862,5543.949951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10813,2024-09-05,654.000000,684.000000,653.700012,664.500000,664.500000,3457002,RITES,Railways,564.500000,30.299988,585.625280,97.873441,986.250000
10814,2024-09-06,667.799988,679.849976,657.099976,661.849976,661.849976,1482454,RITES,Railways,561.849976,22.750000,580.567661,97.869453,998.724976
10815,2024-09-09,665.750000,677.900024,664.000000,672.750000,672.750000,1991325,RITES,Railways,572.750000,13.900024,579.200656,97.885922,1002.125000
10816,2024-09-10,677.000000,684.000000,675.500000,678.900024,678.900024,624747,RITES,Railways,578.900024,8.500000,584.500814,97.895063,1016.450012


In [4]:
df.columns=['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Symbol','Category', 'Daily_Return', 'Price_Range', 'Volatility',
       'Cumulative_Return', 'Average_Price']
df['Date']=pd.to_datetime(df['Date'])
df=df.sort_values(by=['Symbol', 'Date']).reset_index(drop=True)
df=df.drop(columns=['Adj Close', 'Volume'])

In [5]:
print(df.isnull().sum())

Date                 0
Open                 0
High                 0
Low                  0
Close                0
Symbol               0
Category             0
Daily_Return         0
Price_Range          0
Volatility           0
Cumulative_Return    1
Average_Price        0
dtype: int64


In [6]:
df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)


In [7]:
def calculate_sma(data, window):
    return data.rolling(window=window).mean()


def calculate_rsi(data, window=14):
    delta = data.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi


def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    short_ema = data.ewm(span=short_window, adjust=False).mean()
    long_ema = data.ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    return macd - signal


def calculate_volatility(data, window=14):
    return data.pct_change().rolling(window=window).std()


In [8]:
df['SMA_50'] = df.groupby('Symbol')['Close'].transform(lambda x: calculate_sma(x, 50))
df['SMA_200'] = df.groupby('Symbol')['Close'].transform(lambda x: calculate_sma(x, 200))
df['RSI'] = df.groupby('Symbol')['Close'].transform(lambda x: calculate_rsi(x, 14))
df['MACD'] = df.groupby('Symbol')['Close'].transform(lambda x: calculate_macd(x))
df['Volatility'] = df.groupby('Symbol')['Close'].transform(lambda x: calculate_volatility(x, 14))


In [9]:
print(df.head())

        Date         Open         High          Low        Close      Symbol  \
0 2024-01-01  1598.000000  1620.000000  1585.000000  1598.400024  ADANIGREEN   
1 2024-01-02  1598.800049  1616.000000  1565.699951  1603.550049  ADANIGREEN   
2 2024-01-03  1699.000000  1747.400024  1610.000000  1697.250000  ADANIGREEN   
3 2024-01-04  1706.550049  1723.000000  1675.050049  1692.500000  ADANIGREEN   
4 2024-01-05  1695.000000  1700.000000  1660.000000  1674.000000  ADANIGREEN   

  Category  Daily_Return  Price_Range  Volatility  Cumulative_Return  \
0   Energy   1498.400024    35.000000         NaN          27.097994   
1   Energy   1503.550049    50.300049         NaN          27.101216   
2   Energy   1597.250000   137.400024         NaN          27.159648   
3   Energy   1592.500000    47.949951         NaN          27.156850   
4   Energy   1574.000000    40.000000         NaN          27.145919   

   Average_Price  SMA_50  SMA_200  RSI      MACD  
0    2397.200012     NaN      NaN  

In [10]:
df = df.dropna()

In [11]:
df.dtypes

Date                 datetime64[ns]
Open                        float64
High                        float64
Low                         float64
Close                       float64
Symbol                       object
Category                     object
Daily_Return                float64
Price_Range                 float64
Volatility                  float64
Cumulative_Return           float64
Average_Price               float64
SMA_50                      float64
SMA_200                     float64
RSI                         float64
MACD                        float64
dtype: object

In [12]:
df

,Date,Open,High,Low,Close,Symbol,Category,Daily_Return,Price_Range,Volatility,Cumulative_Return,Average_Price,SMA_50,SMA_200,RSI,MACD
2089,2024-05-30,4463.700195,4474.000000,4250.000000,4276.850098,BEML,Railways,4176.850098,224.000000,0.028615,95.560036,6602.125244,3838.675977,3355.864990,56.515190,-50.849910
2090,2024-05-31,4301.799805,4599.899902,4206.649902,4400.649902,BEML,Defence,4300.649902,393.250000,0.015492,76.878397,6502.124756,3854.429976,3363.646740,34.313992,-49.968254
2091,2024-05-31,4301.799805,4599.899902,4206.649902,4400.649902,BEML,Railways,4300.649902,393.250000,0.015492,95.588982,6502.124756,3870.183975,3371.428489,34.313992,-48.752859
2092,2024-06-03,4800.000000,4800.000000,4451.100098,4673.549805,BEML,Defence,4573.549805,348.899902,0.023166,76.940411,7136.774902,3891.440972,3380.878237,58.082157,-29.865043
2093,2024-06-03,4800.000000,4800.000000,4451.100098,4673.549805,BEML,Railways,4573.549805,348.899902,0.023166,95.650996,7136.774902,3912.697969,3390.327986,58.082157,-18.238393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10641,2024-09-10,516.799988,528.000000,514.000000,525.750000,WIPRO,IT,425.750000,14.000000,0.012029,46.243986,779.674988,515.245999,494.391999,38.077246,-1.334465
10642,2024-09-11,525.000000,526.250000,513.250000,514.349976,WIPRO,IT_industry,414.349976,13.000000,0.012968,-0.588325,782.174988,515.742998,494.721999,34.529867,-1.659779
10643,2024-09-11,525.000000,526.250000,513.250000,514.349976,WIPRO,IT,414.349976,13.000000,0.012968,46.222303,782.174988,516.239998,495.051999,34.529867,-1.787209
10644,2024-09-12,520.450012,532.000000,517.750000,530.049988,WIPRO,IT_industry,430.049988,14.250000,0.015555,-0.557801,785.475006,516.892997,495.480499,45.750707,-0.777917


In [13]:
label=LabelEncoder()

In [14]:
df.Symbol=label.fit_transform(df.Symbol).astype('int64')
df.Category=label.fit_transform(df.Category).astype('int64')

In [15]:
X = df.drop(columns=['Close','Date'],axis=1)
y=df['Close']

In [16]:
X

,Open,High,Low,Symbol,Category,Daily_Return,Price_Range,Volatility,Cumulative_Return,Average_Price,SMA_50,SMA_200,RSI,MACD
2089,4463.700195,4474.000000,4250.000000,0,3,4176.850098,224.000000,0.028615,95.560036,6602.125244,3838.675977,3355.864990,56.515190,-50.849910
2090,4301.799805,4599.899902,4206.649902,0,0,4300.649902,393.250000,0.015492,76.878397,6502.124756,3854.429976,3363.646740,34.313992,-49.968254
2091,4301.799805,4599.899902,4206.649902,0,3,4300.649902,393.250000,0.015492,95.588982,6502.124756,3870.183975,3371.428489,34.313992,-48.752859
2092,4800.000000,4800.000000,4451.100098,0,0,4573.549805,348.899902,0.023166,76.940411,7136.774902,3891.440972,3380.878237,58.082157,-29.865043
2093,4800.000000,4800.000000,4451.100098,0,3,4573.549805,348.899902,0.023166,95.650996,7136.774902,3912.697969,3390.327986,58.082157,-18.238393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10641,516.799988,528.000000,514.000000,4,1,425.750000,14.000000,0.012029,46.243986,779.674988,515.245999,494.391999,38.077246,-1.334465
10642,525.000000,526.250000,513.250000,4,2,414.349976,13.000000,0.012968,-0.588325,782.174988,515.742998,494.721999,34.529867,-1.659779
10643,525.000000,526.250000,513.250000,4,1,414.349976,13.000000,0.012968,46.222303,782.174988,516.239998,495.051999,34.529867,-1.787209
10644,520.450012,532.000000,517.750000,4,2,430.049988,14.250000,0.015555,-0.557801,785.475006,516.892997,495.480499,45.750707,-0.777917


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [18]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)
Linear_pred = model.predict(X_test_scaled)
Linear_rmse = np.sqrt(mean_squared_error(y_test, Linear_pred))
r2=r2_score(y_test,Linear_pred)
r2*100

100.0

In [19]:
model_Random = RandomForestRegressor(n_estimators=100, random_state=42)
model_Random.fit(X_train_scaled, y_train)
Random_pred = model_Random.predict(X_test_scaled)
Random_rmse = np.sqrt(mean_squared_error(y_test, Random_pred))
r2=r2_score(y_test,Random_pred)
r2*100


99.99571056850324

In [20]:
xgb = XGBRegressor(n_estimators=100, random_state=42)
xgb.fit(X_train_scaled, y_train)
xgb_pred = xgb.predict(X_test_scaled)
xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_pred))
r2=r2_score(y_test,xgb_pred)
r2*100

99.86844760122163

In [21]:
Linear_rmse,Random_rmse,xgb_rmse

(1.2119417008640767e-12, 11.595317681587003, 64.21436159585296)

In [22]:
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5]
}
grid = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=3, scoring='neg_mean_squared_error')
grid.fit(X_train_scaled, y_train)
print('Best parameters:', grid.best_params_)
print('Best RMSE:', np.sqrt(-grid.best_score_))


Best parameters: {'max_depth': 7, 'min_samples_split': 2, 'n_estimators': 200}
Best RMSE: 24.924457612338266


In [23]:
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.7, 0.8, 1],
    'colsample_bytree': [0.7, 0.8, 1],
}

grid_search = GridSearchCV( estimator=XGBRegressor(random_state=42),param_grid=param_grid,cv=3,scoring='neg_mean_squared_error',
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train_scaled, y_train)
best_model = grid_search.best_estimator_
print('Best parameters:', grid.best_params_)
print('Best RMSE:', np.sqrt(-grid.best_score_))


Fitting 3 folds for each of 162 candidates, totalling 486 fits
Best parameters: {'max_depth': 7, 'min_samples_split': 2, 'n_estimators': 200}
Best RMSE: 24.924457612338266
